## Invoke SageMaker Enpoint from outside of AWS environment using SageMaker SDK

Model used: XGBoost Bike Rental Prediction Trained in the XGBoost Lectures  
  
This example uses the IAM user: ml_user_predict. The user was setup in the housekeeping lecture of the course.  

Refer to the lecture: Configure IAM Users, Setup Command Line Interface (CLI)

Ensure xgboost-biketrain-v1 Endpoint is deployed before running this example  
  
To create an endpoint using SageMaker Console:  
1. Select "Models" under "Inference" in navigation pane
2. Search for model using this prefix: xgboost-biketrain-v1
3. Select the latest model and choose create endpoint
4. Specify endpoint name as: xgboost-biketrain-v1
5. Create a new endpoint configuration
6. Create a new endpoint
7. After this lab is completed, delete the endpoint to avoid unnecessary charges

In [1]:
# Install SageMaker 2.x version.
# !pip install --upgrade sagemaker

     -------------------------------------- 530.1/530.1 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 49.3/49.3 kB 2.4 MB/s eta 0:00:00
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     -------------------------------------- 904.1/904.1 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached importlib_metadata-4.11.4-py3-none-any.whl (18 kB)
     ---------------------------------------- 76.9/76.9 kB 4.4 MB/s eta 0:00:00
  Using cached zipp-3.8.0-py3-none-any.whl (5.4 kB)
     -------------------------------------- 132.3/132.3 kB 7.6 MB/s eta 0:00:00
     ---------------------------------------- 52.5/52.5 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 95.8/95.8 kB 5.3 MB/s eta 0:00:00
  Created wheel for sagemaker: filename=sagemaker-2.95.0-py

In [34]:
import boto3
import sagemaker
import math
import dateutil
import re

# SDK 2 serializers and deserializers
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

In [21]:
# Establish a session with AWS
# Specify credentials and region to be used for this session.
# We will use a ml_user_predict credentials that has limited privileges
boto_session = boto3.Session(profile_name='user_tmd',region_name='us-east-1')

In [22]:
sess = sagemaker.Session(boto_session=boto_session)

In [23]:
# Create a predictor and point to an existing endpoint

# Get Predictor using SageMaker SDK
# Specify Your Endpoint Name
endpoint_name = 'xgboost-biketrain-v1'

predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name,
                                                 sagemaker_session=sess)

In [24]:
# We are sending data for inference in CSV format
predictor.serializer = CSVSerializer()

In [25]:
#datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
# Actual=562
sample_one = '2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027'
# Actual=569
sample_two = '2012-12-19 18:00:00,4,0,1,1,15.58,19.695,50,23.9994'
# Actual=4
sample_three = '2012-12-10 01:00:00,4,0,1,2,14.76,18.94,100,0'

In [26]:
# Raw Data Structure: 
# datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count

# Model expects data in this format (it was trained with these features):
# season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour

def transform_data(data):
    features = data.split(',')
    
    # Extract year, month, day, dayofweek, hour
    dt = dateutil.parser.parse(features[0])

    features.append(str(dt.year))
    features.append(str(dt.month))
    features.append(str(dt.day))
    features.append(str(dt.weekday()))
    features.append(str(dt.hour))
    
    # Return the transformed data. skip datetime field
    return ','.join(features[1:])

In [27]:
print('Raw Data:\n',sample_one)
print('Transformed Data:\n',transform_data(sample_one))

Raw Data:
 2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027
Transformed Data:
 4,0,1,1,16.4,20.455,50,26.0027,2012,12,19,2,17


In [28]:
# Let's invoke prediction now
predictor.predict(transform_data(sample_one))

b'573.6282958984375\n'

In [29]:
# Actual Count is 562...but predicted is 6.3.

# Model was trained with log1p(count)
# So, we need to apply inverse transformation to get the actual count
# Predicted Count looks much better now
result = predictor.predict(transform_data(sample_one))
result = result.decode("utf-8")
print ('Predicted Count', math.expm1(float(result)))

Predicted Count 1.329240521840346e+249


In [30]:
# how to send multiple samples
result = predictor.predict([transform_data(sample_one), transform_data(sample_two)])

In [31]:
result.decode("utf-8")

'573.6282958984375\n547.5216064453125\n'

In [35]:
# Batch Prediction
# Transform data and invoke prediction in specified batch sizes
def run_predictions(data, batch_size):
    predictions = []
    
    transformed_data = [transform_data(row.strip()) for row in data]
    
    for i in range(0, len(data), batch_size):
        
        print(i,i+batch_size)
        
        result = predictor.predict(transformed_data[i : i + batch_size])
        pattern = r'[^0-9.]+'
        re.split(pattern, result.decode())
#         result = result.decode("utf-8")
#         result = result.split(',')
        
        predictions += [math.expm1(float(r)) for r in result]
                
    return predictions

In [36]:
run_predictions([sample_one,sample_two,sample_three],10)

0 10


[1.0413759433029089e+23,
 7.694785265142018e+23,
 1.4093490824269389e+22,
 9.496119420602448e+19,
 2.830753303274694e+23,
 5.184705528587072e+21,
 2.091659496012996e+24,
 5.184705528587072e+21,
 5.685719999335932e+24,
 1.0413759433029089e+23,
 2.091659496012996e+24,
 5.685719999335932e+24,
 2.091659496012996e+24,
 3.831008000716577e+22,
 1.4093490824269389e+22,
 7.694785265142018e+23,
 1.0413759433029089e+23,
 22025.465794806718,
 1.0413759433029089e+23,
 3.831008000716577e+22,
 7.694785265142018e+23,
 9.496119420602448e+19,
 1.0413759433029089e+23,
 5.184705528587072e+21,
 1.9073465724950998e+21,
 2.830753303274694e+23,
 7.016735912097631e+20,
 2.830753303274694e+23,
 3.831008000716577e+22,
 3.831008000716577e+22,
 1.0413759433029089e+23,
 1.4093490824269389e+22,
 1.9073465724950998e+21,
 5.184705528587072e+21,
 1.0413759433029089e+23,
 22025.465794806718,
 1.9073465724950998e+21,
 7.016735912097631e+20,
 9.496119420602448e+19,
 3.831008000716577e+22,
 5.184705528587072e+21,
 1.409349

In [37]:
# Run a batch prediction on Test.CSV File
# Read the file content
data = []
with open('../data/byke sharing/test.csv','r') as f:
    # skip header
    f.readline()
    # Read remaining lines
    data = f.readlines()

In [38]:
len(data)

6493

In [39]:
%%time
predictions = run_predictions(data,500)

0 500
500 1000
1000 1500
1500 2000
2000 2500
2500 3000
3000 3500
3500 4000
4000 4500
4500 5000
5000 5500
5500 6000
6000 6500
CPU times: total: 359 ms
Wall time: 3.8 s


In [40]:
len(predictions),len(data)

(118771, 6493)

In [ ]:
# Don't forget to delete the endpoint
# From SageMaker Console, Select "Endpoints" under Inference and Delete the Endpoint